## Create Validation Test

In [2]:
%load_ext autoreload
%autoreload 2

In [23]:
import sqlite3
import pandas as pd
import numpy as np
from experiment import *

In [4]:
con = sqlite3.connect('../input/database.sqlite')
mobiles = pd.read_sql('select * from devices', con)
desktops = pd.read_sql('select * from cookies', con)
feature_key = 'drawbridge_handle'
mobiles_sample = mobiles.loc[:200000]
mobiles_sample = mobiles_sample[mobiles_sample[feature_key] != '-1']
desktops_sample = desktops.loc[:200000]
desktops_sample = desktops_sample[desktops_sample[feature_key] != '-1']

In [15]:
mobiles_sample = mobiles[mobiles['drawbridge_handle'] != '-1']
print 'Mobiles Size', mobiles_sample.shape
desktops_sample = desktops[desktops['drawbridge_handle'] != '-1']
print 'Desktop Size', desktops_sample.shape

Mobiles Size (142770, 11)
Desktop Size (1643821, 11)


In [18]:
train_data = pd.merge(desktops_sample, mobiles_sample, on='drawbridge_handle', how='inner')[['cookie_id', 'device_id']]

In [19]:
train_data.shape

(180083, 2)

In [20]:
train_data['cookie_id'].value_counts()

id_162004     5
id_4462880    5
id_3086287    5
id_156620     5
id_3908095    5
id_2499943    4
id_114675     4
id_1881853    4
id_2766489    4
id_2224176    4
id_4284912    4
id_3201674    4
id_263203     4
id_4014625    4
id_2260940    4
...
id_4040527    1
id_3049341    1
id_3895624    1
id_3645653    1
id_3876593    1
id_3062955    1
id_870572     1
id_3283196    1
id_578625     1
id_2948822    1
id_3222693    1
id_3222692    1
id_4106921    1
id_2355397    1
id_3552344    1
Length: 175364, dtype: int64

In [21]:
train_data['device_id'].value_counts()

id_257899     28
id_2347997    23
id_1142805    23
id_4110604    22
id_2060336    21
id_2492521    20
id_3225410    19
id_3350627    19
id_4038580    18
id_3133379    18
id_1801122    18
id_4086129    17
id_4688638    17
id_4746684    16
id_4607069    16
...
id_1956637    1
id_1811974    1
id_3331933    1
id_359775     1
id_3097335    1
id_3816821    1
id_551700     1
id_2959153    1
id_1549316    1
id_638649     1
id_1652969    1
id_4297359    1
id_4409023    1
id_157853     1
id_4651261    1
Length: 142770, dtype: int64

We then take 80k sample to product the `local_test` set

In [5]:
'local_x_test.csv'
'local_y_test.csv'

'local_y_test.csv'

In [27]:
train_sample = train_data.loc[np.random.permutation(train_data.index)[:20000]]

In [35]:
device_indices = numpy.unique(train_sample['device_id'])
device_indices

array([u'id_1000317', u'id_1000815', u'id_1001245', ..., u'id_999309',
       u'id_999369', u'id_999787'], dtype=object)

In [69]:
local_test_input = pd.DataFrame({'device_id': device_indices})
local_test_input = pd.merge(mobiles, local_test_input, on='device_id', how='inner')
local_test_input.to_csv('../input/local_test_input.csv', header=True, index=False)
local_test_input.shape

(19354, 11)

In [70]:
local_test_output = pd.DataFrame({'device_id': device_indices})
local_test_output = pd.merge(local_test_output, train_data, on='device_id', how='inner')
local_test_output = local_test_output.groupby('device_id')['cookie_id'].apply(lambda x: "%s" % ' '.join(x)).to_frame().reset_index()
local_test_output.to_csv('../input/local_test_output.csv', header=True, index=False)
local_test_output.shape

(19354, 2)